In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark import SparkConf
from hdfs import InsecureClient
from pyspark.sql.functions import *
from pyspark.sql.functions import from_utc_timestamp, udf, array_distinct, col, when
from pyspark.sql.functions import regexp_replace, year, month, dayofmonth, hour, format_string
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import StringType, TimestampType, DateType, IntegerType
from pyspark.sql.types import DoubleType, StructType, FloatType, StructField
import pandas as pd
import json
import emoji
import stylecloud
from collections import Counter

import warnings

warnings.filterwarnings('ignore') 


In [2]:
spark_conf = SparkConf().setMaster("local[*]").setAppName("Tweets_Hadoop")

spark = SparkSession.builder.config(conf=spark_conf).config('spark.sql.session.timeZone', 'UTC').getOrCreate()

sc = spark.sparkContext

sc.setLogLevel('ERROR')

24/01/02 13:18:44 WARN Utils: Your hostname, BDS-2023 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/01/02 13:18:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/02 13:18:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df_salvo = spark.read.parquet("hdfs://localhost:9000/CA4/sentiment")


In [6]:
#getting sentiment
tweets_df = df_salvo.withColumn('sentiment',when(col("score") > 0, '1').otherwise('0'))

In [7]:
tweets_df.printSchema()

root
 |-- Date/Time: timestamp (nullable = true)
 |-- User: string (nullable = true)
 |-- tweet: string (nullable = true)
 |-- clean_tweet: string (nullable = true)
 |-- prediction: double (nullable = true)
 |-- textblob: float (nullable = true)
 |-- vader: float (nullable = true)
 |-- score: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- sentiment: string (nullable = false)



In [9]:
tweets_fc = tweets_df.withColumn("RT", when(col("tweet").startswith("RT"), 1).otherwise(0))

In [12]:
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

from pyspark.sql.functions import year, month, dayofmonth, hour, col, mean, count, to_date
from pyspark.sql.functions import sum as spark_sum


tweets_hour = tweets_fc.groupBy(year("Date/Time").alias("year"), 
                               month("Date/Time").alias("month"),
                               dayofmonth("Date/Time").alias("day"),
                               to_date(col("Date/Time")).alias("date"),
                               hour("Date/Time").alias("hour"),
                               "sentiment") \
                      .agg(mean(col("score")).alias("hourly_score"),
                           count(col("score")).alias("count_score"),
                           spark_sum(col("RT")).alias("sum_RT"))\
                      .orderBy("year", "month", "day", "hour")
tweets_hour.printSchema()
tweets_hour.show(2)

tweets_hour_b = tweets_fc.groupBy(year("Date/Time").alias("year"), 
                               month("Date/Time").alias("month"),
                               dayofmonth("Date/Time").alias("day"),
                               to_date(col("Date/Time")).alias("date"),
                               hour("Date/Time").alias("hour"),
                               ) \
                      .agg(mean(col("score")).alias("hourly_score"),
                           count(col("score")).alias("count_score"),
                           spark_sum(col("RT")).alias("sum_RT"))\
                      .orderBy("year", "month", "day", "hour")
tweets_hour_b.printSchema()


tweets_hour_b.show(2)

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- hour: integer (nullable = true)
 |-- sentiment: string (nullable = false)
 |-- hourly_score: double (nullable = true)
 |-- count_score: long (nullable = false)
 |-- sum_RT: long (nullable = true)



+----+-----+---+----------+----+---------+--------------------+-----------+------+
|year|month|day|      date|hour|sentiment|        hourly_score|count_score|sum_RT|
+----+-----+---+----------+----+---------+--------------------+-----------+------+
|2009|    5|  2|2009-05-02|   3|        1| 0.34880499728024006|          5|     0|
|2009|    5|  2|2009-05-02|   3|        0|-0.24373822887428104|         10|     0|
+----+-----+---+----------+----+---------+--------------------+-----------+------+
only showing top 2 rows

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- hour: integer (nullable = true)
 |-- hourly_score: double (nullable = true)
 |-- count_score: long (nullable = false)
 |-- sum_RT: long (nullable = true)



+----+-----+---+----------+----+--------------------+-----------+------+
|year|month|day|      date|hour|        hourly_score|count_score|sum_RT|
+----+-----+---+----------+----+--------------------+-----------+------+
|2009|    5|  2|2009-05-02|   3|-0.04622382015610735|         15|     0|
|2009|    5|  2|2009-05-02|   4|-0.14233939349651337|         11|     0|
+----+-----+---+----------+----+--------------------+-----------+------+
only showing top 2 rows



In [13]:
total_tweets = tweets_fc.count()
print(f"Total of Tweets:{total_tweets:,}")

days = tweets_fc.select(to_date(col("Date/Time")).alias("data")).agg(countDistinct("data").alias("total_dias")).first()["total_dias"]
mean_tweets = total_tweets / days
print(f"Average of {mean_tweets:,.0f} tweets per day ({days} total)")

Total of Tweets:1,402


Average of 42 tweets per day (33 total)


In [14]:
tweets_fc = tweets_fc.withColumn("Date/Time", date_format("Date/Time", "yyyy-MM-dd HH:mm:ss")).toPandas()
tweets_fc['entities'] = tweets_fc['entities'].str.replace('\\\\', '\\')

KeyError: 'entities'